# Imports

In [40]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

from typing import Dict

from fake_news.generator_evaluation import evaluateGenerator

from fake_news.classifiers.CNN import ConvolutionalNeuralNetworkClassifier
from fake_news.classifiers.logisticRegression import LogisticRegressionNewsClassifier
from fake_news.classifiers.LSTM import LSTMClassifier
from fake_news.classifiers.NaiveBayes import MultinomialNaiveBayesClassifier
from fake_news.classifiers.RandomForest import RandomForestClassifierClass
from fake_news.classifiers.RNN import RecurrentNeuralNetworkClassifier
from fake_news.classifiers.SVM import SupportVectorMachineClassifier

from fake_news.generators.phi2 import Phi2Generator
from fake_news.generators.t5 import T5Generator
from fake_news.generators.tinyllama import TinyLlamaGenerator

from fake_news.data_preprocessing import DataPreprocessingBeforeClassifiers

from fake_news.base import AbstractNewsGenerator

In [2]:
device = 'cpu' # Change to cuda to use gpu

In [3]:
nltkPath = "C:\\Users\\Kajetan\\AppData\\Roaming\\nltk_data"

In [4]:
seed = 23
np.random.seed(seed)

# Classifier loading

In [5]:
MODEL_PATH = "../fake_news/classifiers"

cnnClassifier = ConvolutionalNeuralNetworkClassifier(metrics=[])
cnnClassifier.load_model(f'{MODEL_PATH}/cnn.keras')

logisticRegressionClassifier = LogisticRegressionNewsClassifier(metrics=[])
logisticRegressionClassifier.load_model(f'{MODEL_PATH}/logisticregression.pkl')

lstmClassifier = LSTMClassifier(metrics=[])
lstmClassifier.load_model(f'{MODEL_PATH}/lstm_model.keras')

naiveBayesClassifier = MultinomialNaiveBayesClassifier(metrics=[])
naiveBayesClassifier.load_model(f'{MODEL_PATH}/naivebayes.pkl')

randomForestClassifier = RandomForestClassifierClass(metrics=[])
randomForestClassifier.load_model(f'{MODEL_PATH}/rf_model.pkl')

rnnClassifier = RecurrentNeuralNetworkClassifier(metrics=[])
rnnClassifier.load_model(f'{MODEL_PATH}/rnn.keras')

svmClassifier = SupportVectorMachineClassifier(metrics=[])
svmClassifier.load_model(f'{MODEL_PATH}/svm_model.pkl')

d:\Users\Kajetan\anaconda3\envs\fake-news\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
d:\Users\Kajetan\anaconda3\envs\fake-news\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 9 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded successfully from: ../fake_news/classifiers/cnn.keras
Model loaded successfully from: ../fake_news/classifiers/logisticregression.pkl
Model loaded successfully from: ../fake_news/classifiers/lstm_model.keras
Model loaded successfully from: ../fake_news/classifiers/naivebayes.pkl


d:\Users\Kajetan\anaconda3\envs\fake-news\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded successfully from: ../fake_news/classifiers/rf_model.pkl
Model loaded successfully from: ../fake_news/classifiers/rnn.keras
Model loaded successfully from: ../fake_news/classifiers/svm_model.pkl


# Generator loading

In [6]:
phi2Generator = Phi2Generator(device_map=device)
t5Generator = T5Generator(device_map=device)
tinyLlamaGenerator = TinyLlamaGenerator(device_map=device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Data loading

In [7]:
df = pd.read_csv('../data/WELFake_clean_test.csv')
df

,Unnamed: 0,title,text,label
0,50990,BLM Rapper & Bill’s Alleged Son Have Nasty Sur...,BLM Rapper & Bill’s Alleged Son Have Nasty Sur...,1
1,41146,US Airstrike Killed Five Al-Qaeda Members in Y...,Get short URL 0 2 0 0 The US military killed f...,1
2,48389,"Zach Johnson, Pieters share lead at Firestone",(Reuters) - Late birdies from Thomas Pieters a...,0
3,55759,Re: WOW! What Josh Earnest admitted about Obam...,WOW! What Josh Earnest admitted about Obamacar...,1
4,21412,Memorial Day provides respite from VA controve...,Memorial Day is a time to remember those who g...,0
...,...,...,...,...
14154,33088,France's Macron unveils plan to curb violence ...,PARIS (Reuters) - French President Emmanuel Ma...,0
14155,31762,HILLARY CLINTON Abruptly Cancels Fundraiser In...,Zero Hedge reported:If there was a time Hillar...,1
14156,45631,"German SPD considers propping up Merkel, but o...",BERLIN (Reuters) - Germany s Social Democrats ...,0
14157,47058,"House Republicans repeal Obamacare, hurdles aw...",WASHINGTON (Reuters) - The U.S. House of Repre...,0


In [8]:
titles = df['title']
titles

0        BLM Rapper & Bill’s Alleged Son Have Nasty Sur...
1        US Airstrike Killed Five Al-Qaeda Members in Y...
2            Zach Johnson, Pieters share lead at Firestone
3        Re: WOW! What Josh Earnest admitted about Obam...
4        Memorial Day provides respite from VA controve...
                               ...                        
14154    France's Macron unveils plan to curb violence ...
14155    HILLARY CLINTON Abruptly Cancels Fundraiser In...
14156    German SPD considers propping up Merkel, but o...
14157    House Republicans repeal Obamacare, hurdles aw...
14158    What Brian Williams’s Chopper Whopper Says Abo...
Name: title, Length: 14159, dtype: object

In [9]:
numOfTitlesToUse = 20

titlesIndices = np.arange(titles.shape[0])
np.random.shuffle(titlesIndices)
titlesIndices = titlesIndices[:numOfTitlesToUse]
titlesIndices

array([11271,  3026, 13315,  7590,  6949,  2108,  9542,  4840, 10540,
       10411,  6039, 12193,  5542, 13449,  7926,  1073,  5516,   329,
       11187,  7780])

In [10]:
titles = titles.iloc[titlesIndices].to_list()
titles

['Final Trump-Clinton debate draws nearly 72 million viewers, third largest ever',
 'Hillary Clinton’s Campaign Used A Green Screen At Today’s Low Turnout Rally',
 "Danger of war, Germany warns after Trump's move on Iran nuclear deal",
 "Philippines' Duterte takes aim at graft agency head for query on his wealth",
 ' RNC’s Hilariously Delusional Election Model Predicts Obama Might Vote Trump',
 'BOXING LEGEND GEORGE FOREMAN Reveals How Trump Saved Him When He Was “Broke…Bankrupt”…BLASTS Athletes Like Kaepernick and Durant For Disrespecting Our Nation',
 'Jerry Jones Violates Law by Treating Police Officers to Cowboys Game - Breitbart',
 'RACHEL MADDOW Announces Plan To Reveal Trump’s Tax Returns…Hey Maddow, you might want to rethink that felony.',
 'BERNIE SANDERS THANKS OBAMA For Not Endorsing A Candidate…Only Hours Later, Obama Endorses Crooked Hillary [VIDEO]',
 'Police arrest 141 in crackdown on North Dakota pipeline protesters',
 ' Nancy Pelosi Reacts To London Attack, Shows Trump

# Data preprocessor loading

In [11]:
dataPreprocessor = DataPreprocessingBeforeClassifiers(nltkPath)
dataPreprocessor.load_tokenizers(f"{MODEL_PATH}/tokenizers/ml_tokenizer.pickle", f"{MODEL_PATH}/tokenizers/dl_tokenizer.pickle")

d:\Users\Kajetan\anaconda3\envs\fake-news\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Users\Kajetan\anaconda3\envs\fake-news\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# Evaluation

In [12]:
classifiersDict = {
    "CNN": (cnnClassifier, "dl"),
    "LogisticRegression": (logisticRegressionClassifier, "ml"),
    "LSTM": (lstmClassifier, "dl"),
    "NaiveBayes": (naiveBayesClassifier, "ml"),
    "RandomForest": (randomForestClassifier, "ml"),
    "RNN": (rnnClassifier, "dl"),
    "SVM": (svmClassifier, "ml"),
}

paramValues = {
    "max_new_tokens": [100, 200],
    "repetition_penalty": [0.5, 1.0],
}

#### Phi2 generator

In [13]:
evaluateGenerator(phi2Generator, classifiersDict, titles, paramValues, dataPreprocessor)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━

CNN  LogisticRegression  LSTM  NaiveBayes  \
max_new_tokens repetition_penalty                                              
100            0.5                 1.0                1.00  1.00        0.95   
               1.0                 0.8                0.55  0.95        0.50   
200            0.5                 1.0                1.00  1.00        0.95   
               1.0                 0.7                0.60  0.85        0.35   

                                   RandomForest   RNN   SVM  
max_new_tokens repetition_penalty                            
100            0.5                         1.00  1.00  1.00  
               1.0                         0.95  0.45  1.00  
200            0.5                         1.00  1.00  1.00  
               1.0                         0.95  0.35  0.95

#### T5 generator

In [14]:
evaluateGenerator(t5Generator, classifiersDict, titles, paramValues, dataPreprocessor)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━

CNN  LogisticRegression  LSTM  NaiveBayes  \
max_new_tokens repetition_penalty                                               
100            0.5                 0.90                0.90  0.90        0.75   
               1.0                 0.90                0.95  0.90        0.80   
200            0.5                 0.90                0.90  0.95        0.75   
               1.0                 0.85                0.90  0.80        0.80   

                                   RandomForest   RNN   SVM  
max_new_tokens repetition_penalty                            
100            0.5                          1.0  0.80  0.60  
               1.0                          1.0  0.80  0.90  
200            0.5                          1.0  0.75  0.65  
               1.0                          1.0  0.70  0.95

#### TinyLlama generator

In [15]:
evaluateGenerator(
    tinyLlamaGenerator, classifiersDict, titles, paramValues, dataPreprocessor
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━

CNN  LogisticRegression  LSTM  NaiveBayes  \
max_new_tokens repetition_penalty                                              
100            0.5                 1.0                1.00  0.95        1.00   
               1.0                 1.0                0.95  1.00        0.90   
200            0.5                 1.0                0.95  0.95        0.95   
               1.0                 1.0                1.00  1.00        1.00   

                                   RandomForest   RNN   SVM  
max_new_tokens repetition_penalty                            
100            0.5                          1.0  1.00  1.00  
               1.0                          1.0  1.00  0.90  
200            0.5                          1.0  0.95  0.95  
               1.0                          1.0  1.00  1.00

# Compare generators outputs to real articles

In [65]:
paramValues = {
    "max_new_tokens": 400,
    "repetition_penalty": 2.0,
}


def compareArticles(generator: AbstractNewsGenerator, paramValues: Dict[str, float]):
    realArticles = df.iloc[titlesIndices].text.to_list()
    generatedArticles = generator.generate(titles, paramValues)
    comparisonDict = {
        'title': [],
        'real': [],
        'generated': []
    }
    for articleId, title in enumerate(titles):
        real = realArticles[articleId]
        generated = generatedArticles[articleId]
        comparisonDict['title'].append(title)
        comparisonDict['real'].append(real)
        comparisonDict['generated'].append(generated)
    comparisonDf = pd.DataFrame(comparisonDict).set_index('title')
    return comparisonDf


#### Phi2 generator

In [79]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(compareArticles(phi2Generator, paramValues))

,real,generated
title,,
"Final Trump-Clinton debate draws nearly 72 million viewers, third largest ever","(Reuters) - An estimated 71.6 million people watched the final debate between White House contenders Hillary Clinton and Donald Trump on TV on Wednesday, below the audience for their first encounter but the third-largest total ever recorded. The data supplied by the Nielsen ratings agency on Thursday covered people who watched Wednesday night’s debate on the four main U.S. broadcast networks plus nine cable and public television channels. The first Clinton-Trump face-off in September attracted a total TV audience of 84 million, the largest in the history of U.S. presidential debates. Last week’s second debate, which was broadcast opposite popular “Sunday Night Football,” was seen by 66.5 million. Wednesday’s audience ranked as the third-highest for a U.S. presidential debate since Nielsen started collecting TV viewership figures for the encounters in 1976. A 1980 debate between Democratic President Jimmy Carter and Republican challenger Ronald Reagan drew 80.6 million viewers. Nielsen data reflects only those who watched the debate on TV at home and did not include millions more who watched online, through social media or in bars and restaurants. The third debate ahead of the Nov. 8 election was marked by Trump’s refusal to commit to accepting the outcome of the election if the Republican candidate loses to his Democratic rival, challenging a cornerstone of American democracy. The presidential election debate, held in Las Vegas, was moderated for the first time by a journalist from Fox News, Chris Wallace. It gave Fox News cable channel the lead in number of viewers for the debate at 11.2 million, followed by broadcasters ABC and NBC with more than 10 million viewers each. Second and third presidential debates have generally attracted smaller TV audiences because many voters have already made up their minds after a presidential campaign lasting more than a year.","The Unexpected Benefits of a Good Night’s Sleep for Your Health and Wellbeing! (Part 1) – The Importance Of A Restful Slumber For Optimal Performance And Productivity At Work By Dr David Heberle | Published on October 31st 2016 Share this post Tweet This Post 0 Comment(0 Likes / 2 Comments ) We all know that getting enough sleep is important to our health but did you also realize how much it can affect your performance at work? In today's article we will discuss the importance behind having good quality rest as well some tips from experts in helping us get better shut eye so let s dive right into what happens when don't have sufficient amounts or poor sleeping habits. What Happens When You Don t Get Enough ZzzzZZZS? According To WebMD, “Sleep deprivation has been linked with an increased risk factors such high blood pressure; heart disease ; diabetes mellitus type II [2]; obesity and depression."" So if these are not reasons alone why one should be concerned about their lack thereof then perhaps knowing more details regarding exactly where they go wrong might help them make changes towards improving themselves before things become worse than just feeling tired during waking hours.. Here Are Some Tips From Experts On How One Can Improve Their Sleeping Habits : - Avoid caffeine after lunchtime If possible avoid drinking coffee past 3pm because its stimulant properties may keep people awake later...even though many believe otherwise....Caffeine stays active within ones system up until 8hours afterwards…so even those who drink decaf could still experience insomnia due simply by consuming caffeinated beverages too late throughout day time Hours Before Bed Time Try reading something relaxing like poetry instead watching TV/movies which tend toward being stimulating It Is Important That People Do Not Use Electronic Devices Like Computers Or Smartphones Right Priorly As mentioned earlier electronic devices emit blue light wavelengths while emitting heat energy making use

#### T5 generator

In [78]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(compareArticles(t5Generator, paramValues))

,real,generated
title,,
"Final Trump-Clinton debate draws nearly 72 million viewers, third largest ever","(Reuters) - An estimated 71.6 million people watched the final debate between White House contenders Hillary Clinton and Donald Trump on TV on Wednesday, below the audience for their first encounter but the third-largest total ever recorded. The data supplied by the Nielsen ratings agency on Thursday covered people who watched Wednesday night’s debate on the four main U.S. broadcast networks plus nine cable and public television channels. The first Clinton-Trump face-off in September attracted a total TV audience of 84 million, the largest in the history of U.S. presidential debates. Last week’s second debate, which was broadcast opposite popular “Sunday Night Football,” was seen by 66.5 million. Wednesday’s audience ranked as the third-highest for a U.S. presidential debate since Nielsen started collecting TV viewership figures for the encounters in 1976. A 1980 debate between Democratic President Jimmy Carter and Republican challenger Ronald Reagan drew 80.6 million viewers. Nielsen data reflects only those who watched the debate on TV at home and did not include millions more who watched online, through social media or in bars and restaurants. The third debate ahead of the Nov. 8 election was marked by Trump’s refusal to commit to accepting the outcome of the election if the Republican candidate loses to his Democratic rival, challenging a cornerstone of American democracy. The presidential election debate, held in Las Vegas, was moderated for the first time by a journalist from Fox News, Chris Wallace. It gave Fox News cable channel the lead in number of viewers for the debate at 11.2 million, followed by broadcasters ABC and NBC with more than 10 million viewers each. Second and third presidential debates have generally attracted smaller TV audiences because many voters have already made up their minds after a presidential campaign lasting more than a year.","ACR s analysis of last night’s debate draws nearly 72 million viewers, third largest ever.... furthermore.... DRIVERS DECLARE FOLLOW THROUGH — Tom Winter (@TimRunsTrump) October 27, 2016 SUPPORT 21WIRE SUBSCRIBE & BECOME A MEMBER @21WIRE.TVnkingly.com/on-air Subscribe to Radiohead Live! Latest News and Specials Coming Soon! https://www.youtube.com/subscription_center?add_user=RussiaToday Like We Are Nownkingly.com/Subscribe-to-NewsNews/show_recently? Follow Us on Facebook! TwitterndikeListening to New Videos Posted by Patrick Henningsen on Oct 28, 2016, 12:52 pm PDTnce Related Itemsding Related Linksndike Read More & Belong With Friends Linked In The Commentsndike Shares Other Storiesding Related Articlesding Related Videosndike Reprintsding Related Videosndike Adsding Related Videosndike Exposes What's Nextndike Topicsding Related Videosndike Infond Related Videosndike Infond Related Videosndike Infond Related Videosndike Infond Related Videosndike Infond Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd Related Videosnd"
Hillary Clinton’s Campaign Used A Green Screen At Today’s Low Turnout Rally,"When you watch this video of Crooked Hillary’s speech from yesterday here in Florida, you will see some amazing things. First, you will see a background section that is in 2-D and not 3-D, like a projected image on a wall would look. Why would that be? Well, that happens in what we call green screen technology, and in this case, poorly done green screen. \nVia EndBegins \n\nWhen you watch this video of Crooked Hillary’s speech from yesterday here in Florida, you will see some amazing things. First, you will see a background section that is in 2-D and 

#### TinyLlama generator

In [80]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(compareArticles(tinyLlamaGenerator, paramValues))

,real,generated
title,,
"Final Trump-Clinton debate draws nearly 72 million viewers, third largest ever","(Reuters) - An estimated 71.6 million people watched the final debate between White House contenders Hillary Clinton and Donald Trump on TV on Wednesday, below the audience for their first encounter but the third-largest total ever recorded. The data supplied by the Nielsen ratings agency on Thursday covered people who watched Wednesday night’s debate on the four main U.S. broadcast networks plus nine cable and public television channels. The first Clinton-Trump face-off in September attracted a total TV audience of 84 million, the largest in the history of U.S. presidential debates. Last week’s second debate, which was broadcast opposite popular “Sunday Night Football,” was seen by 66.5 million. Wednesday’s audience ranked as the third-highest for a U.S. presidential debate since Nielsen started collecting TV viewership figures for the encounters in 1976. A 1980 debate between Democratic President Jimmy Carter and Republican challenger Ronald Reagan drew 80.6 million viewers. Nielsen data reflects only those who watched the debate on TV at home and did not include millions more who watched online, through social media or in bars and restaurants. The third debate ahead of the Nov. 8 election was marked by Trump’s refusal to commit to accepting the outcome of the election if the Republican candidate loses to his Democratic rival, challenging a cornerstone of American democracy. The presidential election debate, held in Las Vegas, was moderated for the first time by a journalist from Fox News, Chris Wallace. It gave Fox News cable channel the lead in number of viewers for the debate at 11.2 million, followed by broadcasters ABC and NBC with more than 10 million viewers each. Second and third presidential debates have generally attracted smaller TV audiences because many voters have already made up their minds after a presidential campaign lasting more than a year.","ixon on twitter today’ s interview with @CNN_Politics about the final #Trump - Clintons ‘defeat' in NY https://t .co/8KoZqxG49P pic.twitter.com /wJlLXgpz1m — CNN (@CNNDebate) December 6 , cnnpoliticalseries Presidential Elections Candidates Donald J Biden and Hillary Rodrıguez de los Estados Unidos en el programa “DeFEAT” http:/www.youtube.c >> Watch more videos at : NPR Politicshas video library Click here to subscribe & login! Sign up for our email list Subscriber Center Show less show full site menu Tweet from ""N PR"" Email Withdrawal of funds after withdrawing $30mn into a new account using an existing debtor (D). What is happening? The fund has been closed by DRAFTING this transaction now… Please SUBSCRIBE TO OUR FREE RESOURCES BY CLICKSY HERE !!! We are also offering special rates if you make one or all your donations via PayPal – click below link FOR INFO ON PASSWORD SAVE AND REQUESTS …..We will be adding these specials as time goes forward so check back soon!!! If not please contact us through Facebook ..I am sorry but we do NOT accept payments made directly offline !!Please feel free to use any means possible such as text messaging etc that doesnot require payment whatsoever This includes credit cards OR anything else which requires human interaction like Paisa Bank deposits online only!)Click Here For More Information About Our Free Programmes ...CLOSURE OF PROJECT IMPORTANT UPDATE :- After several days following publication of last weekend 'Operation Mockup', Project Veritas announced yesterday morning they would no longer publish their investigative"
Hillary Clinton’s Campaign Used A Green Screen At Today’s Low Turnout Rally,"When you watch this video of Crooked Hillary’s speech from yesterday here in Florida, you will see some amazing things. First, you will see a background section that is in 2-D and not 3-D, like a projected image on a wall would look. Why would that be? Well, that happens in what we call green screen technology, and in this case,